In [1]:
import pymysql
import pandas as pd
import time

In [2]:
# create conneciton with the database
def create_connection(h, u, pw, db):
    connection = pymysql.connect(host = h,
                                 user = u,
                                 password = pw,
                                 database = db,
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    return connection

# Insert tweets 

In [3]:
# read the tweets from the file 
tweet_df = pd.read_csv("tweet.csv", sep = "\t").set_index("index")
tweet_df.head(5)

,tweet_id,user_id,tweet_ts,tweet_text
index,,,,
1,3021907,1003874,2021-01-19 10:48:17,%d~&$[MU)&DOU%eCc_Gu*?q(X@vVb%NaPk)YrP-(GY%S>U...
2,4138357,1008782,2021-01-11 04:34:05,"dSxBkJfXs$P;fNZ[(KLGFJ""c!s|^o$rInI':h^~t@]NeB*..."
3,7827189,1001835,2021-01-21 23:03:27,vG:bJcepa>-f?}kSum%hZu!w{+I[M{]UBwqLDh/)XtAmRl...
4,5163060,1005198,2021-01-12 04:11:05,"i!.A/X,Xo|IwM-z^'H!m@)'+\U{=Py|_x#\iTcO)!%ztO{..."
5,4575657,1003600,2021-01-15 21:47:34,"bFvaxjmuaG+~r#?|ETPYX""{@|GaXRLP?l^/;cj$T/k'KmM..."


In [4]:
# prepare the query to insert the record
sql = "INSERT INTO `tweets` (tweet_id, user_id, tweet_ts, tweet_text) VALUES (%s, %s, %s, %s)"

# insert one record at a time
def insert_one_tweet(values, cursor):
    cursor.execute(sql, tuple(values))    

In [5]:
# number of the row
row_num = len(tweet_df.index)

def insert_all_tweets(row_num):
    
    connection = create_connection("localhost", "root", "Hjm123456", "twitter")
    # create a cursor object
    cursor = connection.cursor()
    
    # the start time of inserting the tweet
    start_time = time.time()
        
    for i in range(row_num):
        row = [str(item) for item in list(tweet_df.iloc[i])]
        
        try: # insert one record
            insert_one_tweet(row, cursor)
        except:
            connection.rollback()
            
    # the end time of inserting the tweet        
    end_time = time.time()       
        
    # commit update    
    connection.commit()
    connection.close()
        
    print('Time used to insert', row_num, 'tweets:', end_time - start_time, 'seconds.')

In [6]:
insert_all_tweets(row_num)

Time used to insert 1000000 tweets: 397.83448791503906 seconds.


# Insert Followers

In [7]:
# read the followers from the file 
follower_df = pd.read_csv("follower.csv", sep = "\t").set_index("index")
follower_df.head(5)

,user_id,follows_id
index,,
1,1003874,1002776
2,1003874,1007457
3,1003874,1008323
4,1003874,1005300
5,1003874,1003598


In [8]:
# prepare the query to insert the record
sql = "INSERT INTO `followers` (user_id, follows_id) VALUES (%s, %s)"

# insert one record at a time
def insert_one_follower(values, cursor):
    cursor.execute(sql, tuple(values))

In [9]:
# number of the row
row_num = len(follower_df.index)

def insert_all_followers(row_num):
    
    connection = create_connection("localhost", "root", "Hjm123456", "twitter")
    # create a cursor object
    cursor = connection.cursor()
    
    # the start time of inserting the tweet
    start_time = time.time()
        
    j = 0    
    for i in range(row_num):
        row = [str(item) for item in list(follower_df.iloc[i])]
        
        try: # insert one record
            insert_one_follower(row, cursor)
            
        except:
            connection.rollback()

            
    # the end time of inserting the tweet        
    end_time = time.time()       
        
    # commit update    
    connection.commit()
    connection.close()
        
    print('Time used to insert', row_num, 'followers:', end_time - start_time, 'seconds.')

In [10]:
insert_all_followers(row_num)

Time used to insert 970000 followers: 398.10966992378235 seconds.


# Get Timeline

In [60]:
import random

# get num_timeline timelines, with or without tweet_text for n users
def get_random_timeline(n, bol, num_timeline):
    
    connection = create_connection("localhost", "root", "Hjm123456", "twitter")
    # create a cursor object
    cursor = connection.cursor()
    
    sql_start_with_text = '''SELECT 
                             tweet_id, tweet_ts, tweet_text 
                             FROM tweets 
                             WHERE tweets.user_id IN
                             (    SELECT follows_id
                             FROM followers
                          '''
    
    sql_start_without_text = '''SELECT 
                                tweet_id, tweet_ts 
                                FROM tweets 
                                WHERE tweets.user_id IN
                                (    SELECT follows_id
                                FROM followers
                             '''
    
    
    sql_end = '''  ) 
                   ORDER BY tweet_ts DESC
                   LIMIT 
              ''' + str(num_timeline)
    
    total_time = 0
    
    for i in range (n):
        
        # randomly pick a user
        random_user_id = random.randint(1000001, 1010000)
        if bol: # select with tweet_text
            sql = sql_start_with_text + "WHERE user_id = " + str(random_user_id) + sql_end
        else: # select without tweet_text
            sql = sql_start_without_text + "WHERE user_id = " + str(random_user_id) + sql_end
    
        start_time = time.time()
        cursor.execute(sql)
        end_time = time.time()
        
        total_time += (end_time - start_time)
    
    print("Time used to get", n, "random users' timeline:", total_time, "seconds")
    print("The average time to get one random user's timeline is", round(total_time / n, 5), "seconds")

In [62]:
num_user_list = [1, 10, 30, 50]

In [55]:
# Only retrieve tweet_id and tweet_ts (10 most recent)
for num in num_user_list:
    get_random_timeline(num, False, 10)

Time used to get 1 random users' timeline: 0.3390007019042969 seconds
The average time to get one random user's timeline is 0.339 seconds
Time used to get 10 random users' timeline: 3.248065233230591 seconds
The average time to get one random user's timeline is 0.32481 seconds
Time used to get 30 random users' timeline: 9.603729724884033 seconds
The average time to get one random user's timeline is 0.32012 seconds
Time used to get 50 random users' timeline: 16.310139417648315 seconds
The average time to get one random user's timeline is 0.3262 seconds


In [57]:
# Retrieve tweet_id, tweet_ts, and tweet_text (10 most recent)
for num in num_user_list:
    get_random_timeline(num, True, 10)

Time used to get 1 random users' timeline: 0.3545668125152588 seconds
The average time to get one random user's timeline is 0.35457 seconds
Time used to get 10 random users' timeline: 3.3163557052612305 seconds
The average time to get one random user's timeline is 0.33164 seconds
Time used to get 30 random users' timeline: 9.645811319351196 seconds
The average time to get one random user's timeline is 0.32153 seconds
Time used to get 50 random users' timeline: 16.285800457000732 seconds
The average time to get one random user's timeline is 0.32572 seconds


In [67]:
# Only retrieve tweet_id and tweet_ts (20 most recent)
for num in num_user_list:
    get_random_timeline(num, False, 20)

Time used to get 1 random users' timeline: 0.3271350860595703 seconds
The average time to get one random user's timeline is 0.32714 seconds
Time used to get 10 random users' timeline: 3.3278422355651855 seconds
The average time to get one random user's timeline is 0.33278 seconds
Time used to get 30 random users' timeline: 10.189840316772461 seconds
The average time to get one random user's timeline is 0.33966 seconds
Time used to get 50 random users' timeline: 16.799941301345825 seconds
The average time to get one random user's timeline is 0.336 seconds


In [68]:
# Retrieve tweet_id, tweet_ts, and tweet_text (20 most recent)
for num in num_user_list:
    get_random_timeline(num, True, 20)

Time used to get 1 random users' timeline: 0.37426090240478516 seconds
The average time to get one random user's timeline is 0.37426 seconds
Time used to get 10 random users' timeline: 3.539879560470581 seconds
The average time to get one random user's timeline is 0.35399 seconds
Time used to get 30 random users' timeline: 10.786177396774292 seconds
The average time to get one random user's timeline is 0.35954 seconds
Time used to get 50 random users' timeline: 17.027554512023926 seconds
The average time to get one random user's timeline is 0.34055 seconds
